Data cleanup
---

This notebook takes the results from the data acquisition stage and merges them into a pair of files, one for tweet data and one for user data.

The cleaned data is saved in the [merged_data](./merged_data/)

This creates a couple large files `/merged_data/merged_tweets.csv` and `/merged_data/merged_users.csv`. These should be ran on a local machine and `.gitignore`d. They are not included in this repo.

The tweets and user data are merged in the [following notebook featuring EDA](./3_EDA).


### Import libraries

In [3]:
import pandas as pd
import numpy as np
import os
import re
import pickle

#######################################################
# change this when importing newly gathered tweet data
#######################################################
max_csv_number = 193

## Merging and saving tweet data

Go through each tweets or users csv in `raw_data`, concatenate them, and save the result.


### get_raw_data
Reads a single csv and returns a dataframe.

### get_merged_data
Repeatedly calls `get_raw_data` and merges all of the dataframes into one.

In [4]:
merged_data_path = './merged_data/'
raw_data_path = './raw_data/'

#loads the data in a csv and returns a dataframe with it.
# can get tweet data or user data.
def get_raw_data(kind = 'tweets', number = 0):
    new_df = pd.read_csv(f'{raw_data_path}{kind}_{number}.csv')
    new_df.set_index('id', inplace = True)
    return new_df


# repeatedly call the previous, tacking each onto the end of a new dataframe,
# and return the dataframe
def get_merged_data(kind = 'tweets', numbers = [0,1]):
    #make a new blank df
    new_df = pd.DataFrame()
    #grab each file and tack data onto the end of the dataframe
    for number in numbers:
        this_df = get_raw_data(kind = kind, number = number)
        if kind == 'tweets':
            new_df = pd.concat([new_df, this_df])
        else: #users or users_idfixed
            #explicitly making it 'outer' may be unneccesary. Just making sure
            # there are no duplicates.
            new_df = pd.concat([new_df, this_df], join = 'outer')
    #convert created_at string to datetime before returning
    if 'created_at' in new_df.columns:
        new_df['created_at'] = pd.to_datetime(new_df['created_at'])
    return new_df


In [5]:
%%time

#perform the merge for all saved tweet csvs
tweets_df = get_merged_data(kind = 'tweets', numbers = range(max_csv_number + 1))

Wall time: 19.3 s


In [6]:
#examine the top of the results.
tweets_df['text'].head()

id
1521733018293084160    Wordle (ES)  #118 5/6\n\n⬜⬜🟨🟩⬜\n⬜⬜🟨🟩⬜\n⬜⬜⬜🟩🟩\n...
1521733015738662913    Wordle Científico #52 5/6\n\n⬜⬜⬜⬜🟨\n🟨⬜⬜⬜🟩\n⬜🟩⬜...
1521733011414257664         Wordle 319 4/6\n\n🟨🟨⬜⬜⬜\n🟨⬜⬜🟩⬜\n🟩🟨⬜🟩🟨\n🟩🟩🟩🟩🟩
1521733010504445954    Back in the saddle.\n\nWordle 319 3/6\n\n⬜🟩⬜🟨⬜...
1521733001981444096    @RwellsWells Same. Are we both spotters? Haha ...
Name: text, dtype: object

In [7]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 386457 entries, 1521733018293084160 to 1521994876677672966
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype              
---  ------               --------------   -----              
 0   conversation_id      386457 non-null  int64              
 1   reply_settings       386457 non-null  object             
 2   lang                 386457 non-null  object             
 3   entities             123247 non-null  object             
 4   possibly_sensitive   386457 non-null  bool               
 5   source               386457 non-null  object             
 6   public_metrics       386457 non-null  object             
 7   text                 386457 non-null  object             
 8   context_annotations  381900 non-null  object             
 9   author_id            386457 non-null  int64              
 10  created_at           386457 non-null  datetime64[ns, UTC]
 11  referenced_tweets    40081 non-nul

Dropping duplicate IDs with the help of [This stack overflow question.](https://stackoverflow.com/questions/13035764/remove-pandas-rows-with-duplicate-indices)

In [8]:
tweets_df = tweets_df[~tweets_df.index.duplicated(keep = 'first')]
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 335327 entries, 1521733018293084160 to 1521994876677672966
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype              
---  ------               --------------   -----              
 0   conversation_id      335327 non-null  int64              
 1   reply_settings       335327 non-null  object             
 2   lang                 335327 non-null  object             
 3   entities             107512 non-null  object             
 4   possibly_sensitive   335327 non-null  bool               
 5   source               335327 non-null  object             
 6   public_metrics       335327 non-null  object             
 7   text                 335327 non-null  object             
 8   context_annotations  331475 non-null  object             
 9   author_id            335327 non-null  int64              
 10  created_at           335327 non-null  datetime64[ns, UTC]
 11  referenced_tweets    35202 non-nul

In [9]:
%%time

#save the merged data
tweets_df.to_csv(merged_data_path + 'merged_tweets.csv')

Wall time: 8.15 s


Now do the same for the users.

## Merging and saving user data


### Fixing user data ids

The user data needs to be edited a bit and resaved. The `Unnamed: 0` column is partly indices 0-99, and the index is id, while later they switch. In fact, the ids are all NaNs later on! 

It does seem like it's not reliably always exactly the first 100 with these indices, so I will make sure to check appropriately.

- for each user_xx.csv,
    - pull it in
    - check through all non-NaN indices. for each,
        - copy the index into the Unnamed: 0 column
        - reindex on Unnamed: 0
        - verify that all indices are unique, nothing is lost
        - save an updated copy

In [10]:
# fix user indexes.
# pulls in `users_{number}.csv, fixes the index so that it is the user id, and saves a new copy.

raw_data_path = './raw_data/'
old_user_data_path = raw_data_path


def fix_user_index(number, extension = '_idfixed'):
    this_user_df = pd.read_csv(old_user_data_path + f'users_{number}.csv', index_col=None)
    this_user_df['id'] = this_user_df.apply(lambda row: row['Unnamed: 0'] if row.isna()['id'] else row['id'], axis = 1).astype(np.int64)
    this_user_df.set_index('id', inplace = True)
    this_user_df.drop(columns = 'Unnamed: 0', inplace = True)
    #save in the raw data path. do not overwrite the originals.
    this_user_df.to_csv(raw_data_path + f'users{extension}_{number}.csv')
    return None
    

In [11]:
%%time
# fix all of the user data and save new csvs
first_users_number = 0
last_users_number = max_csv_number
for user_number in range(first_users_number, last_users_number + 1):
    fix_user_index(user_number)
print("done!")

done!
Wall time: 45.7 s


### Merging and saving id-fixed user data

This procedure is identical as that for the tweets data.

In [12]:
%%time
users_df = get_merged_data(kind = 'users_idfixed', numbers = range(max_csv_number+1))

Wall time: 22.7 s


In [13]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 383910 entries, 1006110502043471872 to 125443746
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   name               383897 non-null  object 
 1   public_metrics     383910 non-null  object 
 2   url                117064 non-null  object 
 3   description        336144 non-null  object 
 4   profile_image_url  383910 non-null  object 
 5   location           272627 non-null  object 
 6   protected          383910 non-null  bool   
 7   username           383910 non-null  object 
 8   verified           383910 non-null  bool   
 9   entities           161563 non-null  object 
 10  pinned_tweet_id    181222 non-null  float64
 11  withheld           4 non-null       object 
dtypes: bool(2), float64(1), object(9)
memory usage: 33.0+ MB


In [14]:
#drop duplicates. ~... is the list of users to keep, the ones that are NOT duplicates
users_df = users_df[~users_df.index.duplicated(keep = 'first')]
users_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147313 entries, 1006110502043471872 to 233970658
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   name               147306 non-null  object 
 1   public_metrics     147313 non-null  object 
 2   url                44780 non-null   object 
 3   description        129085 non-null  object 
 4   profile_image_url  147313 non-null  object 
 5   location           105401 non-null  object 
 6   protected          147313 non-null  bool   
 7   username           147313 non-null  object 
 8   verified           147313 non-null  bool   
 9   entities           62081 non-null   object 
 10  pinned_tweet_id    68630 non-null   float64
 11  withheld           1 non-null       object 
dtypes: bool(2), float64(1), object(9)
memory usage: 12.6+ MB


In [15]:
# save merged user data
users_df.to_csv(merged_data_path + 'merged_users.csv')

# Next step: EDA